# Maria's results

In [1]:
import pandas as pd
import numpy as np

## Load data

In [28]:
transporters = pd.read_excel("input/files_from_fairdomhub/metabolism_summary.xlsx",sheet_name="Transporters")

energy = pd.read_excel("input/files_from_fairdomhub/metabolism_summary.xlsx",sheet_name="Energy")

organic_nitrogen = pd.read_excel("input/files_from_fairdomhub/metabolism_summary.xlsx",sheet_name="Organic Nitrogen")

In [36]:
all_mags_paper = pd.read_excel("input/files_from_fairdomhub/All_Mags_for_paper_analysis.xlsx",sheet_name="Coverage")
all_mags_paper.set_index("MAG",inplace=True)

## Transporters

In [29]:
transporters

,gene_id,gene_description,module,header,subheader,CH1-bin.9,CH8-bin.29,CH8-bin.9,CH8-bin.16,CH13-bin.6,...,CH15-bin.6,CH15-bin.3,CH13-bin.24,CH1-bin.1,CH2-bin.7,CH15-bin.15,CH7-bin.20,CH13-bin.16,CH8-bin.25,CH8-bin.0
0,K06073,"ABC.VB12.P, btuC; vitamin B12 transport system...",Cobalamin salvage,Vitamin B12 transport system,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,K06074,"ABC.VB12.A, btuD; vitamin B12 transport system...",Cobalamin salvage,Vitamin B12 transport system,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,K06858,"ABC.VB12.S1, btuF; vitamin B12 transport syste...",Cobalamin salvage,Vitamin B12 transport system,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,K16092,btuB; vitamin B12 transporter,Cobalamin salvage,Vitamin B12 transport system,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,K11081,phnS; 2-aminoethylphosphonate transport system...,2-Aminoethylphosphonate transport system,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,K12340,tolC; outer membrane channel protein TolC,alpha-Hemolysin/cyclolysin transport system,NaN,NaN,1,2,0,3,0,...,1,0,2,4,1,0,0,2,1,1
645,K02065,"mlaF, linL, mkl; phospholipid/cholesterol/gamm...",gamma-Hexachlorocyclohexane transport system,NaN,NaN,3,2,0,1,0,...,2,0,0,1,3,2,0,0,1,1
646,K02066,"mlaE, linK; phospholipid/cholesterol/gamma-HCH...",gamma-Hexachlorocyclohexane transport system,NaN,NaN,3,2,0,1,0,...,2,0,0,1,3,2,0,0,1,1
647,K02067,"mlaD, linM; phospholipid/cholesterol/gamma-HCH...",gamma-Hexachlorocyclohexane transport system,NaN,NaN,3,1,0,0,0,...,2,0,0,1,3,2,0,0,1,1


In [142]:
def find_transport_coverage(compound):
    # Remove uninteresting columns
    transporters_sub =transporters.loc[:,~transporters.columns.isin(["header","subheader"])]

    # Find transporters genes related to this compound
    transporters_sub = transporters_sub[transporters_sub.gene_description.str.contains(compound)]
    # Set gene information as index
    transporters_sub.set_index(["gene_id","module"],inplace=True)
    transporters_sub = transporters_sub.groupby("gene_description").mean()
    
    total_possible_genes = len(transporters_sub.index)
    
    # Only include MAGs that have at least one of these genes
    transporters_sub = transporters_sub.sum()[transporters_sub.sum()>0]

    # Calculate fraction of total genes that they have
    frac_transporters_sub = transporters_sub/total_possible_genes
    frac_transporters_sub.name = "fraction_of_possible_transporters"
    
    frac_transporters_sub = pd.concat([all_mags_paper.loc[:,["Source","Substrate","Phylum","Coverage (%)"]],frac_transporters_sub],axis=1).dropna()
    frac_transporters_sub.index.name = "MAG"
    frac_transporters_sub = frac_transporters_sub.reset_index().set_index(["Source","Substrate"]).sort_index()
    
    return frac_transporters_sub

#### Sulfate

In [143]:
find_transport_coverage("sulfate")

MAG             Phylum  Coverage (%)  \
Source            Substrate                                                 
Compost_Digestate Avicel     CH13-bin.14       Firmicutes_A         18.25   
                  PASC       CH15-bin.16       Firmicutes_A          7.44   
                  PASC       CH15-bin.22   Desulfobacterota          4.90   
                  PASC       CH15-bin.18       Firmicutes_A          1.03   
                  Xylan       CH14-bin.1       Firmicutes_A         56.45   
                  Xylan       CH14-bin.2       Firmicutes_A         21.05   
Cow_Manure        Avicel      CH8-bin.17  Verrucomicrobiota          8.69   
                  Avicel       CH8-bin.6  Verrucomicrobiota          4.37   
                  PASC         CH7-bin.2  Verrucomicrobiota          2.87   
                  PASC        CH7-bin.25  Verrucomicrobiota          0.38   
                  Xylan        CH9-bin.1       Firmicutes_A          5.83   
Marshland         PASC         CH1-bin.6       Firmicutes_A         56.11   
                  PASC         CH1-bin.7     Proteobacteria          1.84   
                  Xylan        CH3-bin.2       Firmicutes_A         74.31   
                  Xylan        CH3-bin.0         Firmicutes          0.81   

                             fraction_of_possible_transporters  
Source            Substrate                                     
Compost_Digestate Avicel                                   0.8  
                  PASC                                     0.6  
                  PASC                                     0.8  
                  PASC                                     0.8  
                  Xylan                                    0.6  
                  Xylan                                    0.8  
Cow_Manure        Avicel                                   0.8  
                  Avicel                                   0.8  
                  PASC                                     0.8  
                  PASC                                     0.6  
                  Xylan                                    0.8  
Marshland         PASC                                     0.6  
                  PASC                                     0.4  
                  Xylan                                    0.8  
                  Xylan                                    0.4

#### Sulfonate

In [149]:
df = find_transport_coverage("sulfonate")
df.sort_values(by="fraction_of_possible_transporters",ascending=False).head(20)

MAG              Phylum  Coverage (%)  \
Source            Substrate                                                  
Marshland         Xylan        CH3-bin.2        Firmicutes_A         74.31   
Compost_Digestate Avicel     CH13-bin.14        Firmicutes_A         18.25   
                  PASC       CH15-bin.18        Firmicutes_A          1.03   
                  PASC       CH15-bin.17  Desulfobacterota_I          5.21   
                  Avicel      CH13-bin.1  Desulfobacterota_I          1.60   
Marshland         PASC        CH1-bin.10      Proteobacteria          0.80   
                  PASC         CH1-bin.7      Proteobacteria          1.84   
                  PASC         CH1-bin.0        Firmicutes_A          0.29   
Compost_Digestate Avicel      CH13-bin.2        Firmicutes_A          1.10   
                  Xylan       CH14-bin.2        Firmicutes_A         21.05   
                  PASC       CH15-bin.22    Desulfobacterota          4.90   
Marshland         Xylan        CH3-bin.0          Firmicutes          0.81   
Cow_Manure        Xylan        CH9-bin.1        Firmicutes_A          5.83   
Marshland         Xylan        CH3-bin.1        Firmicutes_A         21.93   
Cow_Manure        Xylan        CH9-bin.0    Actinobacteriota         65.10   
Marshland         PASC         CH1-bin.6        Firmicutes_A         56.11   
Compost_Digestate PASC       CH15-bin.20        Firmicutes_A          0.73   
Marshland         PASC         CH1-bin.8        Firmicutes_A          4.81   
Compost_Digestate Avicel      CH13-bin.0        Firmicutes_A          4.37   
Cow_Manure        Avicel      CH8-bin.20       Chloroflexota          0.45   

                             fraction_of_possible_transporters  
Source            Substrate                                     
Marshland         Xylan                               3.666667  
Compost_Digestate Avicel                              3.000000  
                  PASC                                2.166667  
                  PASC                                1.833333  
                  Avicel                              1.833333  
Marshland         PASC                                1.666667  
                  PASC                                1.666667  
                  PASC                                1.500000  
Compost_Digestate Avicel                              1.500000  
                  Xylan                               1.333333  
                  PASC                                1.333333  
Marshland         Xylan                               1.333333  
Cow_Manure        Xylan                               1.166667  
Marshland         Xylan                               1.166667  
Cow_Manure        Xylan                               1.000000  
Marshland         PASC                                1.000000  
Compost_Digestate PASC                                1.000000  
Marshland         PASC                                1.000000  
Compost_Digestate Avicel                              1.000000  
Cow_Manure        Avicel                              1.000000

### Synthetic pathways for organic nitrogen compounds

In [163]:
def find_nitrogen_pathway_coverage(compound):
    organic_nitrogen_sub = organic_nitrogen.loc[:,~organic_nitrogen.columns.isin(["subheader","header"])]
    organic_nitrogen_sub = organic_nitrogen_sub[~organic_nitrogen_sub["module"].isna()]

    organic_nitrogen_sub = organic_nitrogen_sub[organic_nitrogen_sub["module"].str.contains(compound)]
    organic_nitrogen_sub_mean = organic_nitrogen_lys.set_index(["gene_id","module"]).groupby("gene_description").mean()
    count_sub = len(organic_nitrogen_sub_mean.index)

    organic_nitrogen_sub_frac = organic_nitrogen_sub_mean.sum()[organic_nitrogen_sub_mean.sum()>0]/count_sub

    organic_nitrogen_sub_frac.index.name = "MAG"
    organic_nitrogen_sub_frac.name = "fraction_of_pathway"
    organic_nitrogen_sub_frac = pd.concat([all_mags_paper.loc[:,["Source","Substrate","Phylum","Coverage (%)"]],organic_nitrogen_sub_frac],axis=1).dropna()

    organic_nitrogen_sub_frac = organic_nitrogen_sub_frac.reset_index().set_index(["Source","Substrate"]).sort_index()
    
    return organic_nitrogen_sub_frac

In [172]:
lysine_df = find_nitrogen_pathway_coverage("Lysine biosynthesis")
lysine_df

MAG              Phylum  Coverage (%)  \
Source            Substrate                                                  
Compost_Digestate Avicel     CH13-bin.12      Fibrobacterota         28.82   
                  Avicel     CH13-bin.14        Firmicutes_A         18.25   
                  Avicel      CH13-bin.4        Bacteroidota         10.41   
                  Avicel     CH13-bin.11  Desulfobacterota_I          7.87   
                  Avicel     CH13-bin.25       Spirochaetota          6.52   
...                                  ...                 ...           ...   
Marshland         PASC         CH1-bin.5        Firmicutes_A          0.44   
                  PASC         CH1-bin.0        Firmicutes_A          0.29   
                  Xylan        CH3-bin.2        Firmicutes_A         74.31   
                  Xylan        CH3-bin.1        Firmicutes_A         21.93   
                  Xylan        CH3-bin.0          Firmicutes          0.81   

                             fraction_of_pathway  
Source            Substrate                       
Compost_Digestate Avicel                0.223684  
                  Avicel                0.468623  
                  Avicel                0.321862  
                  Avicel                0.231781  
                  Avicel                0.387652  
...                                          ...  
Marshland         PASC                  0.284413  
                  PASC                  0.266869  
                  Xylan                 0.355263  
                  Xylan                 0.223684  
                  Xylan                 0.284413  

[100 rows x 4 columns]

In [176]:
lysine_df[lysine_df.fraction_of_pathway>0.3]

MAG              Phylum  Coverage (%)  \
Source            Substrate                                                  
Compost_Digestate Avicel     CH13-bin.14        Firmicutes_A         18.25   
                  Avicel      CH13-bin.4        Bacteroidota         10.41   
                  Avicel     CH13-bin.25       Spirochaetota          6.52   
                  Avicel      CH13-bin.0        Firmicutes_A          4.37   
                  Avicel     CH13-bin.10        Firmicutes_A          0.70   
                  PASC        CH15-bin.1        Bacteroidota         10.26   
                  PASC       CH15-bin.23        Firmicutes_A          9.73   
                  PASC        CH15-bin.7       Spirochaetota          7.73   
                  PASC       CH15-bin.16        Firmicutes_A          7.44   
                  PASC        CH15-bin.5        Bacteroidota          3.62   
                  PASC       CH15-bin.12        Bacteroidota          2.11   
                  PASC       CH15-bin.18        Firmicutes_A          1.03   
                  PASC       CH15-bin.20        Firmicutes_A          0.73   
                  Xylan       CH14-bin.1        Firmicutes_A         56.45   
                  Xylan       CH14-bin.2        Firmicutes_A         21.05   
Cow_Manure        Avicel      CH8-bin.17   Verrucomicrobiota          8.69   
                  Avicel      CH8-bin.25  Desulfobacterota_I          6.12   
                  Avicel       CH8-bin.6   Verrucomicrobiota          4.37   
                  Avicel      CH8-bin.16        Bacteroidota          1.64   
                  Avicel      CH8-bin.29        Bacteroidota          0.98   
                  Avicel       CH8-bin.0        Firmicutes_C          0.60   
                  Avicel      CH8-bin.27        Firmicutes_A          0.45   
                  PASC         CH7-bin.8       Spirochaetota          3.47   
                  PASC         CH7-bin.2   Verrucomicrobiota          2.87   
                  PASC        CH7-bin.17    Desulfobacterota          2.06   
                  PASC        CH7-bin.20        Firmicutes_A          1.59   
                  PASC         CH7-bin.6        Bacteroidota          0.94   
                  PASC        CH7-bin.15       Spirochaetota          0.86   
                  PASC        CH7-bin.14    Desulfobacterota          0.61   
                  PASC        CH7-bin.25   Verrucomicrobiota          0.38   
                  Xylan        CH9-bin.1        Firmicutes_A          5.83   
                  Xylan        CH9-bin.5          Firmicutes          2.74   
                  Xylan        CH9-bin.4          Firmicutes          1.87   
Marshland         PASC         CH1-bin.6        Firmicutes_A         56.11   
                  PASC         CH1-bin.8        Firmicutes_A          4.81   
                  PASC         CH1-bin.7      Proteobacteria          1.84   
                  PASC        CH1-bin.10      Proteobacteria          0.80   
                  Xylan        CH3-bin.2        Firmicutes_A         74.31   

                             fraction_of_pathway  
Source            Substrate                       
Compost_Digestate Avicel                0.468623  
                  Avicel                0.321862  
                  Avicel                0.387652  
                  Avicel                0.358974  
                  Avicel                0.403509  
                  PASC                  0.321862  
                  PASC                  0.345816  
                  PASC                  0.387652  
                  PASC                  0.389676  
                  PASC                  0.321862  
                  PASC                  0.321862  
                  PASC                  0.521255  
                  PASC                  0.380904  
                  Xylan                 0.363360  
                  Xylan                 0.363360  
Cow_Manure        Avicel                0.337045 

In [177]:
find_nitrogen_pathway_coverage("Cysteine biosynthesis")

MAG              Phylum  Coverage (%)  \
Source            Substrate                                                  
Compost_Digestate Avicel     CH13-bin.12      Fibrobacterota         28.82   
                  Avicel     CH13-bin.14        Firmicutes_A         18.25   
                  Avicel      CH13-bin.4        Bacteroidota         10.41   
                  Avicel     CH13-bin.11  Desulfobacterota_I          7.87   
                  Avicel     CH13-bin.25       Spirochaetota          6.52   
...                                  ...                 ...           ...   
Marshland         PASC         CH1-bin.5        Firmicutes_A          0.44   
                  PASC         CH1-bin.0        Firmicutes_A          0.29   
                  Xylan        CH3-bin.2        Firmicutes_A         74.31   
                  Xylan        CH3-bin.1        Firmicutes_A         21.93   
                  Xylan        CH3-bin.0          Firmicutes          0.81   

                             fraction_of_pathway  
Source            Substrate                       
Compost_Digestate Avicel                0.223684  
                  Avicel                0.468623  
                  Avicel                0.321862  
                  Avicel                0.231781  
                  Avicel                0.387652  
...                                          ...  
Marshland         PASC                  0.284413  
                  PASC                  0.266869  
                  Xylan                 0.355263  
                  Xylan                 0.223684  
                  Xylan                 0.284413  

[100 rows x 4 columns]

In [178]:
find_nitrogen_pathway_coverage("Tryptophan biosynthesis")

MAG              Phylum  Coverage (%)  \
Source            Substrate                                                  
Compost_Digestate Avicel     CH13-bin.12      Fibrobacterota         28.82   
                  Avicel     CH13-bin.14        Firmicutes_A         18.25   
                  Avicel      CH13-bin.4        Bacteroidota         10.41   
                  Avicel     CH13-bin.11  Desulfobacterota_I          7.87   
                  Avicel     CH13-bin.25       Spirochaetota          6.52   
...                                  ...                 ...           ...   
Marshland         PASC         CH1-bin.5        Firmicutes_A          0.44   
                  PASC         CH1-bin.0        Firmicutes_A          0.29   
                  Xylan        CH3-bin.2        Firmicutes_A         74.31   
                  Xylan        CH3-bin.1        Firmicutes_A         21.93   
                  Xylan        CH3-bin.0          Firmicutes          0.81   

                             fraction_of_pathway  
Source            Substrate                       
Compost_Digestate Avicel                0.223684  
                  Avicel                0.468623  
                  Avicel                0.321862  
                  Avicel                0.231781  
                  Avicel                0.387652  
...                                          ...  
Marshland         PASC                  0.284413  
                  PASC                  0.266869  
                  Xylan                 0.355263  
                  Xylan                 0.223684  
                  Xylan                 0.284413  

[100 rows x 4 columns]